In [1]:
import os

In [2]:
train_dir = "/kaggle/input/firedataset/dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train"
test_dir = "/kaggle/input/firedataset/dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/test"

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [4]:
batch_size = 32
img_size = (224, 224)

train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size
)

test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size
)

# Tối ưu hóa pipeline (prefetch)
AUTOTUNE = tf.data.AUTOTUNE
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

Found 31703 files belonging to 3 classes.
Found 9484 files belonging to 3 classes.


In [5]:
from tensorflow.keras.regularizers import l2

# Tải MobileNetV2 với trọng số ImageNet
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

# Đóng băng các lớp của base model
base_model.trainable = False

# Thêm các lớp tùy chỉnh
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Lấy giá trị trung bình toàn cục
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)  # Dropout để giảm overfitting
x = Dense(128, activation="relu")(x)  # Lớp ẩn
predictions = Dense(3, activation="softmax")(x)  # Lớp đầu ra (3 class)

# Tạo mô hình hoàn chỉnh
model = Model(inputs=base_model.input, outputs=predictions)

# Compile mô hình
model.compile(optimizer=Adam(learning_rate=0.001),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# Huấn luyện mô hình
epochs = 10
history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs)


In [ ]:
from sklearn.metrics import classification_report

# Dự đoán trên tập test
y_pred = model.predict(test_dataset)
y_pred_classes = y_pred.argmax(axis=1)  # Lấy class dự đoán (với softmax)

# Lấy nhãn thật
y_true = test_dataset.labels  

# In báo cáo chi tiết
print(classification_report(y_true, y_pred_classes, target_names=["fire", "non fire", "Smoke"]))
